In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 49.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=fb4d86b6a97c4a15414456a7c91aef24c1e59ed878f7e97a76b710689ebe5462
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:

import os
import pyspark
from pyspark.sql import SparkSession
conf = pyspark.SparkConf()
#sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","10g")\
        .getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
person = spark.read.option("header",True).option("inferSchema", True).csv("/content/drive/MyDrive/NewYork Dataset/Motor_Vehicle_Collisions_-_Person.csv")

In [5]:
vehicle = spark.read.option("header",True).option("inferSchema", True).csv("/content/drive/MyDrive/NewYork Dataset/Motor_Vehicle_Collisions_-_Vehicles.csv")

In [6]:
crash = spark.read.option("header",True).option("inferSchema", True).csv("/content/drive/MyDrive/NewYork Dataset/Motor_Vehicle_Collisions_-_Crashes.csv")

In [7]:
crash_filtered=crash.na.drop(subset=["CRASH DATE","LATITUDE","LONGITUDE"])

In [8]:
crash_filtered.show()

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUT

In [9]:
crash.count()

1888285

In [10]:
crash.printSchema()

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [11]:
from pyspark.sql.functions import date_format
from pyspark.sql.functions import unix_timestamp
import pyspark.sql.functions as F

crash_filtered=crash_filtered.withColumn('year',(date_format(unix_timestamp("CRASH DATE", "MM/dd/yyyy").cast("timestamp"), "yyyy")))
crash_filtered=crash_filtered.withColumn('month',(date_format(unix_timestamp("CRASH DATE", "MM/dd/yyyy").cast("timestamp"), "MM")))

In [12]:
crash_filtered.groupBy('year').agg(F.count('COLLISION_ID')).show()

+----+-------------------+
|year|count(COLLISION_ID)|
+----+-------------------+
|2016|             192616|
|2020|             103990|
|2012|              85458|
|2019|             194113|
|2017|             217084|
|2018|             216445|
|2022|              30148|
|2021|             101731|
|null|                  0|
|2015|             182979|
|2014|             172738|
|2013|             171934|
+----+-------------------+



In [13]:

crash_filtered=crash_filtered.filter(crash_filtered.LONGITUDE!=0.0)
crash_filtered=crash_filtered.filter(crash_filtered.LATITUDE!=0.0)
yearly_crash_count=crash_filtered.groupBy('year').agg(F.count('COLLISION_ID').alias("yearly_crash_count"))
monthly_crash_count=crash_filtered.groupBy('year','month').agg(F.count('COLLISION_ID').alias("monthly_crash_count"))

Monthly and Yearly Crash Counts

In [14]:
yearly_crash_count.show(20)
monthly_crash_count.show(20)

+----+------------------+
|year|yearly_crash_count|
+----+------------------+
|2016|            192181|
|2020|            103828|
|2012|             85458|
|2019|            193819|
|2017|            216992|
|2018|            216150|
|2022|             29204|
|2021|            100857|
|2015|            182979|
|2014|            172738|
|2013|            171933|
+----+------------------+

+----+-----+-------------------+
|year|month|monthly_crash_count|
+----+-----+-------------------+
|2020|   11|               8326|
|2020|   10|               8912|
|2018|   07|              18528|
|2019|   04|              15385|
|2018|   04|              17086|
|2021|   10|               9293|
|2018|   10|              19070|
|2021|   09|               8914|
|2016|   04|               7777|
|2019|   09|              16153|
|2019|   07|              16943|
|2019|   11|              15481|
|2021|   07|               9212|
|2019|   08|              15802|
|2020|   03|              10187|
|2018|   08|   

In [15]:
crash_filtered.show()

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+--------------------+-------------------+-------------------+----+-----+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLE

In [16]:
#Table

import math
def getBins(val):
  val=float(val)
  bin=math.floor(val/0.3)*0.3
  return bin

getBinsUDF=pyspark.sql.functions.udf(getBins)

In [17]:
crash_filtered_temp=crash_filtered.withColumn('latBin',getBinsUDF('LATITUDE'))
crash_filtered_temp=crash_filtered_temp.withColumn('longBin',getBinsUDF('LONGITUDE'))

In [18]:
crash_filtered_temp.select(F.countDistinct('latBin','longBin')).show()

+-------------------------------+
|count(DISTINCT latBin, longBin)|
+-------------------------------+
|                             20|
+-------------------------------+



In [19]:
crash_filtered_temp.select('*').show()

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+--------------------+-------------------+-------------------+----+-----+------+------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURE

In [20]:
contributing_factor_filtered = crash_filtered.filter((F.col('CONTRIBUTING FACTOR VEHICLE 1') != 'null') & (F.col('CONTRIBUTING FACTOR VEHICLE 1') != '1') & (F.col('CONTRIBUTING FACTOR VEHICLE 1') != '80'))

In [21]:
contributing_factor_filtered.select('CONTRIBUTING FACTOR VEHICLE 1').distinct().count()

59

Prime Contributing Factor and it's Count

In [22]:
#Prime contributing factor
prime_contributory_cause_df=contributing_factor_filtered.groupBy('year','month','CONTRIBUTING FACTOR VEHICLE 1').agg(F.count('CONTRIBUTING FACTOR VEHICLE 1').alias('prim_contributory_cause_count'))
prime_contributory_cause_df = prime_contributory_cause_df.withColumnRenamed("CONTRIBUTING FACTOR VEHICLE 1","contributing_factor_vehicle_1")
prime_contributory_cause_df.show()

+----+-----+-----------------------------+-----------------------------+
|year|month|contributing_factor_vehicle_1|prim_contributory_cause_count|
+----+-----+-----------------------------+-----------------------------+
|2021|   02|         Traffic Control D...|                            6|
|2020|   12|               Animals Action|                            5|
|2020|   11|         Traffic Control D...|                            2|
|2021|   08|         Lane Marking Impr...|                            4|
|2020|   06|              Fatigued/Drowsy|                           10|
|2021|   10|                  Fell Asleep|                           46|
|2021|   11|         Unsafe Lane Changing|                          167|
|2020|   02|          Driver Inexperience|                          170|
|2020|   01|                 Unsafe Speed|                          211|
|2018|   09|         View Obstructed/L...|                          142|
|2018|   04|         Failure to Yield ...|         

In [23]:
newyork_crash_summary_df = crash_filtered_temp.groupBy('year','month','latBin','longBin')\
.agg(F.count(F.col('COLLISION_ID')).alias("crash_count"),\
     F.sum(F.col('NUMBER OF PERSONS INJURED')).alias("injuries_count"),\
     F.sum(F.col('NUMBER OF PERSONS KILLED')).alias("kill_count"))

New York Crash Summary

In [24]:
newyork_crash_summary_df.show()

+----+-----+------+-------------------+-----------+--------------+----------+
|year|month|latBin|            longBin|crash_count|injuries_count|kill_count|
+----+-----+------+-------------------+-----------+--------------+----------+
|2019|   06|  40.8|              -74.1|       3667|        1134.0|         3|
|2019|   08|  40.5|              -73.8|       1579|         532.0|         0|
|2020|   07|  40.5|              -74.1|       5356|        2498.0|        15|
|2018|   06|  40.5|-201.29999999999998|          6|           2.0|         0|
|2020|   05|  40.5| -74.39999999999999|        187|          60.0|         1|
|2018|   11|  40.5|              -73.8|       1770|         539.0|         5|
|2021|   06|  40.8|              -73.8|          2|           1.0|         0|
|2018|   09|  40.5| -74.39999999999999|        691|         163.0|         0|
|2018|   07|  40.8|              -73.8|         18|           0.0|         0|
|2018|   07|  40.5|              -74.1|      12249|        3286.

In [25]:
vehicle = vehicle.withColumn('VEHICLE_TYPE',F.when(vehicle.VEHICLE_TYPE=="Bus","BUS")\
                             .when(vehicle.VEHICLE_TYPE=="LARGE COM VEH(6 OR MORE TIRES)","BUS")\
                             .when(vehicle.VEHICLE_TYPE=="Van","VAN")\
                             .when(vehicle.VEHICLE_TYPE=="Van","VAN")\
                             .when(vehicle.VEHICLE_TYPE=="Taxi","TAXI")\
                             .when(vehicle.VEHICLE_TYPE=="PASSENGER VEHICLE","TAXI")\
                             .when(vehicle.VEHICLE_TYPE=="LIVERY VEHICLE","TAXI")\
                             .when(vehicle.VEHICLE_TYPE=="Motorcycle","MOTORCYCLE")\
                             .when(vehicle.VEHICLE_TYPE=="4 dr sedan","Sedan")\
                             .when(vehicle.VEHICLE_TYPE=="Convertible","Sedan")\
                             .when(vehicle.VEHICLE_TYPE=="SMALL COM VEH(4 TIRES)","Sedan")\
                             .when(vehicle.VEHICLE_TYPE=="Station Wagon/Sport Utility Vehicle","Sedan")\
                             .when(vehicle.VEHICLE_TYPE=="Pick-up Truck","Truck")\
                             .when(vehicle.VEHICLE_TYPE=="PICK-UP TRUCK","Truck")\
                             .when(vehicle.VEHICLE_TYPE=="Dump","Truck")\
                             .when(vehicle.VEHICLE_TYPE=="Tractor Truck Diesel","Truck")\
                             .when(vehicle.VEHICLE_TYPE=="Box Truck","Truck")\
                             .otherwise(vehicle.VEHICLE_TYPE))

In [26]:
vehicle.show()

+---------+------------+----------+----------+--------------------+------------------+------------+--------------+-------------+------------+----------------+-----------------+----------+---------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+------------------+----------------+----------------------+---------------------------+---------------------+---------------------+
|UNIQUE_ID|COLLISION_ID|CRASH_DATE|CRASH_TIME|          VEHICLE_ID|STATE_REGISTRATION|VEHICLE_TYPE|  VEHICLE_MAKE|VEHICLE_MODEL|VEHICLE_YEAR|TRAVEL_DIRECTION|VEHICLE_OCCUPANTS|DRIVER_SEX|DRIVER_LICENSE_STATUS|DRIVER_LICENSE_JURISDICTION|           PRE_CRASH|     POINT_OF_IMPACT|      VEHICLE_DAMAGE|    VEHICLE_DAMAGE_1|  VEHICLE_DAMAGE_2|VEHICLE_DAMAGE_3|PUBLIC_PROPERTY_DAMAGE|PUBLIC_PROPERTY_DAMAGE_TYPE|CONTRIBUTING_FACTOR_1|CONTRIBUTING_FACTOR_2|
+---------+------------+----------+----------+--------------------+------------------+--------

In [27]:
vehicle=vehicle.withColumn('year',(date_format(unix_timestamp("CRASH_DATE", "MM/dd/yyyy").cast("timestamp"), "yyyy")))
vehicle=vehicle.withColumn('month',(date_format(unix_timestamp("CRASH_DATE", "MM/dd/yyyy").cast("timestamp"), "MM")))
vehicle_type_count=vehicle.groupBy('year','VEHICLE_TYPE').agg(F.count('VEHICLE_TYPE').alias('count'))
vehicle_type_count=vehicle_type_count.filter(F.col('count')>5000)
vehicle_type_count=vehicle_type_count.filter(F.col('VEHICLE_TYPE')!='UNKNOWN')
vehicle_type_count.count()

57

Vehicle Types with it's Crash Count

In [28]:
vehicle_type_count.show()

+----+--------------------+------+
|year|        VEHICLE_TYPE| count|
+----+--------------------+------+
|2019|                 BUS|  6721|
|2016|                Bike|  6054|
|2014|                TAXI|226669|
|2014|SPORT UTILITY / S...| 90028|
|2013|               Truck|  6061|
|2013|               OTHER| 11233|
|2012|                 VAN|  6454|
|2018|               Sedan|353298|
|2020|                Bike|  5317|
|2017|                 BUS|  7404|
|2014|                 VAN| 13417|
|2019|                Bike|  6045|
|2016|                 VAN|  6144|
|2012|               OTHER|  5423|
|2017|                TAXI| 19347|
|2015|                 VAN| 14072|
|2016|               Sedan|284305|
|2016|               Truck| 22607|
|2015|                 BUS| 16044|
|2018|               Truck| 29434|
+----+--------------------+------+
only showing top 20 rows



In [29]:
import json
def rdd_json_to_dict(input_json):
  output_dict = [json.loads(record) for record in input_json]
  return output_dict

In [ ]:
newyork_crash_summary_dict = rdd_json_to_dict(newyork_crash_summary_df.toJSON().collect())
yearly_crash_count_dict=rdd_json_to_dict(yearly_crash_count.toJSON().collect())
monthly_crash_count_dict=rdd_json_to_dict(monthly_crash_count.toJSON().collect())
prime_contributory_cause_dict=rdd_json_to_dict(prime_contributory_cause_df.toJSON().collect())
vehicle_type_count_dict=rdd_json_to_dict(vehicle_type_count.toJSON().collect())

In [ ]:
from pymongo import MongoClient
import os
MONGO_URI = os.environ["MONGO_URI"]
client = MongoClient(MONGO_URI)
db = client["TrafficAnalysis"]
coll = db["newyork_crash_summary"]
coll.insert_many(newyork_crash_summary_dict)
coll = db['newyork_year_crash_count']
coll.insert_many(yearly_crash_count_dict)
coll=db['newyork_monthly_crash_count']
coll.insert_many(monthly_crash_count_dict)
coll=db['newyork_prime_contributory_cause']
coll.insert_many(prime_contributory_cause_dict)
coll=db['newyork_vehicle_type_crash_cause']
coll.insert_many(vehicle_type_count_dict)